In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [62]:
df = pd.read_csv("a_steam_data_2021_2025.csv")

In [49]:
df.head()

,appid,name,release_year,release_date,genres,categories,price,recommendations,developer,publisher
0,3057270,Seafarer's Gambit,2024,"Jul 5, 2024",Action;Adventure;Indie;RPG;Strategy,Single-player;Family Sharing,3.99,0,Bouncy Rocket Studios,Bouncy Rocket Studios
1,3822840,Capitalist Misadventures,2025,"Jul 25, 2025",Casual;Indie;Simulation;Strategy,Single-player;Save Anytime;Family Sharing,7.99,0,Caramelo Studios,Caramelo Studios
2,3216640,The Beast and the Princess,2025,"Jun 17, 2025",Adventure;Indie;Strategy,Single-player;Steam Achievements;Full controll...,12.99,0,Libragames,Libragames
3,2403620,Air Twister,2023,"Nov 10, 2023",Action;Adventure;Indie,Single-player;Steam Achievements;Full controll...,24.99,0,YS Net,ININ
4,1538040,Horde Slayer,2021,"Mar 19, 2021",Action;Adventure;Casual;Indie;RPG;Early Access,Single-player;Steam Achievements;Full controll...,3.99,0,Wagner Rodrigues,Wagner Rodrigues


In [4]:
df.describe()

,appid,release_year,price,recommendations
count,6.552100e+04,65521.000000,65521.000000,65521.000000
mean,2.580607e+06,2023.453091,7.204043,362.165336
std,7.692480e+05,1.386641,16.914429,6936.837198
min,1.200220e+06,2021.000000,0.000000,0.000000
25%,1.916970e+06,2022.000000,0.990000,0.000000
50%,2.516050e+06,2024.000000,3.990000,0.000000
75%,3.193800e+06,2025.000000,9.740000,0.000000
max,4.243930e+06,2025.000000,1900.000000,862487.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65521 entries, 0 to 65520
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   appid            65521 non-null  int64  
 1   name             65521 non-null  object 
 2   release_year     65521 non-null  int64  
 3   release_date     65521 non-null  object 
 4   genres           65455 non-null  object 
 5   categories       65514 non-null  object 
 6   price            65521 non-null  float64
 7   recommendations  65521 non-null  int64  
 8   developer        65468 non-null  object 
 9   publisher        65338 non-null  object 
dtypes: float64(1), int64(3), object(6)
memory usage: 5.0+ MB


In [7]:
df.nunique()

appid              65521
name               65128
release_year           5
release_date        1828
genres              1723
categories          7993
price                589
recommendations     2536
developer          45679
publisher          41600
dtype: int64

In [8]:
df.isnull().sum()

appid                0
name                 0
release_year         0
release_date         0
genres              66
categories           7
price                0
recommendations      0
developer           53
publisher          183
dtype: int64

In [38]:
average_revenue_for_publisher = df.groupby('publisher')['price'].mean()
print(average_revenue_for_publisher.sort_values(ascending=False).head(10))

publisher
FiatLux Corporatoin             1900.00
A&S Inc.                         999.98
Zekertopia                       615.38
Whoes heart broken               500.00
Azerbaijan Technology;Aztech     199.99
ArchaeologyGames                 199.99
chen yuang                       199.99
Kele whd                         199.99
 Rawi Studio                     199.99
Remax                            199.99
Name: price, dtype: float64


In [36]:
revenue_for_publisher = df.groupby('publisher')['price'].sum()
print(revenue_for_publisher.sort_values(ascending=False).head(10))

publisher
Hede                            26478.59
Rawi Studio                      2800.85
ArchaeologyGames                 2199.89
Azerbaijan Technology;Aztech     2199.89
BFG Entertainment                2143.88
Square Enix                      2091.55
A&S Inc.                         1999.96
FiatLux Corporatoin              1900.00
SEGA                             1845.50
Kagura Games                     1784.22
Name: price, dtype: float64


In [37]:
count_for_publisher = df.groupby('publisher')['appid'].count()
print(count_for_publisher.sort_values(ascending=False).head(10))

publisher
EroticGamesClub                    215
BFG Entertainment                  172
8floor                             150
Hede                               142
Kagura Games                       120
Bully Revenge Studios              119
DigiPen Institute of Technology    100
Gamesforgames                       98
indie.io                            93
Do Games Limited                    90
Name: appid, dtype: int64


In [42]:
count_for_genres = df['genres'].value_counts()
print(count_for_genres.head(10))

genres
Casual;Indie               4136
Action;Adventure;Indie     2883
Action;Indie               2632
Adventure;Indie            2325
Casual                     2196
Adventure;Casual;Indie     2056
Action;Casual;Indie        1703
Indie                      1672
Action                     1631
Casual;Indie;Simulation    1456
Name: count, dtype: int64


Making changes with release_date column, converting it into datetime object in python:

Checking what problem do I have:

In [64]:
df['release_date_original'] = df['release_date']
df['release_date'] = pd.to_datetime(df['release_date'], format="%b %d, %Y", errors='coerce')

In [65]:
invalid_dates = df[df['release_date'].isna()]

print("Number of invalid dates:", len(invalid_dates))
print(invalid_dates[['release_date']].head())

Number of invalid dates: 1400
    release_date
10           NaT
28           NaT
36           NaT
72           NaT
123          NaT


In [66]:
invalid_dates = df[df['release_date'].isna()]
print(invalid_dates[['release_date_original']])

      release_date_original
10                  Q4 2025
28                  Q4 2025
36                  Q4 2025
72                     2025
123                    2025
...                     ...
65194               Q4 2025
65211               Q4 2025
65289                  2025
65312                  2025
65492                  2025

[1400 rows x 1 columns]


In [69]:
# Function to handle invalid dates
def parse_release_date(date_str):
    if pd.isna(date_str):
        return pd.NaT

    s = str(date_str).strip().upper()

    # Handle quarter formats: Q4 2025 or Q42025
    if s.startswith("Q"):
        q = s[1]                 # quarter number
        year = s[-4:]            # last 4 chars are year
        try:
            period = pd.Period(f"{year}Q{q}", freq="Q")
            return period.start_time
        except:
            return pd.NaT

    # Handle year-only format
    if s.isdigit() and len(s) == 4:
        return pd.to_datetime(f"{s}-01-01")

    # Handle full dates
    return pd.to_datetime(s, errors="coerce")


In [70]:
df['release_date'] = df['release_date_original'].apply(parse_release_date)


In [76]:
df.columns.tolist()

['appid',
 'name',
 'release_year',
 'release_month',
 'release_day',
 'genres',
 'categories',
 'price',
 'recommendations',
 'developer',
 'publisher']

In [72]:
df['release_month'] = df['release_date'].dt.month.astype('Int64')
df['release_day']   = df['release_date'].dt.day.astype('Int64')

In [73]:
df.drop(columns=['release_date_original', 'release_date'], inplace=True)

In [75]:
df = df[
    [
        'appid',
        'name',
        'release_year',
        'release_month',
        'release_day',
        'genres',
        'categories',
        'price',
        'recommendations',
        'developer',
        'publisher'
    ]
]
